<a href="https://colab.research.google.com/github/Launcha10/sports_entertaiment_topic_analysis/blob/hayashi-test/travel_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

!pip install gensim
!pip install wordcloud 
!pip install pyLDAvis

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-l

In [0]:
import MeCab

In [4]:
tagger = MeCab.Tagger("-Owakati")
sample = "私はご飯を食べました"
print(tagger.parse(sample))

私 は ご飯 を 食べ まし た 



In [0]:
import glob
import numpy as np
import random
from tqdm import tqdm
import math
import MeCab
import urllib
import gensim
import pyLDAvis
import pyLDAvis.gensim
from wordcloud import WordCloud
from PIL import Image
import matplotlib
import matplotlib.pylab as plt

from gensim import corpora, models, matutils

np.random.seed(0)
FONT = "/content/drive/My Drive/ernie/travel_data/font/ipag.ttf"

In [0]:
# paths to textss
text_paths = glob.glob('/content/drive/My Drive/ernie/travel_data/new_text/new_text2/*.txt')

# define stop words
req = urllib.request.Request('http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt')
with urllib.request.urlopen(req) as res:
    stopwords = res.read().decode('utf-8').split('\r\n')
while '' in stopwords:
    stopwords.remove('')

In [0]:
def analyzer(text, mecab, stopwords=[], target_part_of_speech=['proper_noun', 'noun', 'verb', 'adjective']):

    node = mecab.parseToNode(text)
    words = []

    while node:

        features = node.feature.split(',')
        surface = features[6]

        if (surface == '*') or (len(surface) < 2) or (surface in stopwords):
            node = node.next
            continue

        noun_flag = (features[0] == '名詞')
        proper_noun_flag = (features[0] == '名詞') & (features[1] == '固有名詞')
        verb_flag = (features[0] == '動詞') & (features[1] == '自立')
        adjective_flag = (features[0] == '形容詞') & (features[1] == '自立')


        if ('proper_noun' in target_part_of_speech) & proper_noun_flag:
            words.append(surface)
        elif ('noun' in target_part_of_speech) & noun_flag:
            words.append(surface)
        elif ('verb' in target_part_of_speech) & verb_flag:
            words.append(surface)
        elif ('adjective' in target_part_of_speech) & adjective_flag:
            words.append(surface)

        node = node.next

    return words

In [0]:
import re
#無関係なワード削除
def format_text(text):

    text=re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-…]+', "", text)
    #text=re.sub('投稿', "", text)
    text=re.sub(r'[!-~]', "", text)#半角記号,数字,英字
    text=re.sub(r'[︰-＠]', "", text)#全角記号
    #text=re.sub('\n', " ", text)#改行文字

    return text

In [9]:
# make dictionary and corpus
mecab = MeCab.Tagger('-Owakati')
titles = []
texts = []
for text_path in text_paths:

    text = open(text_path, 'r').read()
    text = text.split('\n')
    title = text[2]
    text = ' '.join(text[3:])
    #print(text)
    format_words = format_text(text)
    words = analyzer(format_words, mecab, stopwords=stopwords, target_part_of_speech=['noun', 'proper_noun'])
    #print(words)
    texts.append(words)
    #print(texts)

dictionary = gensim.corpora.Dictionary(texts)
print(dictionary.token2id)
dictionary.filter_extremes(no_below=3, no_above=0.5)
print(dictionary)
corpus = [dictionary.doc2bow(t) for t in texts]
print(corpus)

{'あか': 0, 'あさり': 1, 'あずき': 2, 'あちこち': 3, 'あつあつ': 4, 'あふれ': 5, 'あま': 6, 'あまり': 7, 'あれこれ': 8, 'あん': 9, 'あんこ': 10, 'いかが': 11, 'いくら': 12, 'いずみ': 13, 'いずれ': 14, 'いち': 15, 'いちご': 16, 'いっぱい': 17, 'いつか': 18, 'いも': 19, 'うぐいす色': 20, 'うさぎ': 21, 'うどん': 22, 'うまみ': 23, 'うるう': 24, 'えびす': 25, 'おい': 26, 'おかず': 27, 'おから': 28, 'おこ': 29, 'おしゃべり': 30, 'おしゃれ': 31, 'おじさん': 32, 'おすすめ': 33, 'おすそ分け': 34, 'おとめ': 35, 'おなか': 36, 'おもてなし': 37, 'おん': 38, 'お世話': 39, 'お供': 40, 'お供え': 41, 'お出かけ': 42, 'お参り': 43, 'お墨付き': 44, 'お子さん': 45, 'お子様': 46, 'お守り': 47, 'お客': 48, 'お客様': 49, 'お家': 50, 'お寺': 51, 'お座敷': 52, 'お昼': 53, 'お気': 54, 'お気に入り': 55, 'お洒落': 56, 'お湯': 57, 'お目見え': 58, 'お腹': 59, 'お茶': 60, 'お茶屋': 61, 'お菓子': 62, 'お話': 63, 'お金': 64, 'お鍋': 65, 'お隣': 66, 'かき氷': 67, 'かけ': 68, 'かしこ': 69, 'かな': 70, 'かなり': 71, 'かぼちゃ': 72, 'かわり': 73, 'かん': 74, 'がかり': 75, 'がち': 76, 'がん': 77, 'きっかけ': 78, 'きつね': 79, 'きなこ': 80, 'きまぐれ': 81, 'きれい': 82, 'きん': 83, 'くし': 84, 'くだもの': 85, 'くちばし': 86, 'くど': 87, 'くに': 88, 'くま': 89, 'くりくり': 90, 'くるみ': 91, '

In [10]:
print(type(corpus))

<class 'list'>


In [12]:
#bowではなく、tfidfを試す
"""

import os
import gc
import logging
import pickle
from smart_open import open

# tfidf
tfidf = gensim.models.TfidfModel(corpus)
tfidf.save('model.tfidf')

# make corpus_tfidf
corpus_tfidf = tfidf[corpus]

# save corpus_tfidf
with open("corpus_tfidf.pkl",'wb') as f:
    pickle.dump(corpus_tfidf,f)

print(corpus_tfidf)

"""

'\n\nimport os\nimport gc\nimport logging\nimport pickle\nfrom smart_open import open\n\n# tfidf\ntfidf = gensim.models.TfidfModel(corpus)\ntfidf.save(\'model.tfidf\')\n\n# make corpus_tfidf\ncorpus_tfidf = tfidf[corpus]\n\n# save corpus_tfidf\nwith open("corpus_tfidf.pkl",\'wb\') as f:\n    pickle.dump(corpus_tfidf,f)\n\nprint(corpus_tfidf)\n\n'

In [0]:
# KL divergenceの定義と描画
def calc_topic_distances(m, topic):
    import numpy as np

    def kldiv(p, q):
        distance = np.sum(p * np.log(p / q))
        return distance

    # get probability of each words
    # https://github.com/piskvorky/gensim/blob/develop/gensim/models/ldamodel.py#L733

    t = m.state.get_lambda()
    for i, p in enumerate(t):
        t[i] = t[i] / t[i].sum()

    base = t[topic]
    distances = [(i_p[0], kldiv(base, i_p[1])) for i_p in enumerate(t) if i_p[0] != topic]
    return distances

def plot_distance_matrix(m):
    import numpy as np
    import matplotlib.pylab as plt

    # make distance matrix
    mt = []
    for i in range(m.num_topics):
        d = calc_topic_distances(m, i)
        d.insert(i, (i, 0))  # distance between same topic
        d = [_d[1] for _d in d]
        mt.append(d)

    mt = np.array(mt)

    # plot matrix
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.set_aspect("equal")
    plt.imshow(mt, interpolation="nearest", cmap=plt.cm.ocean)
    plt.yticks(range(mt.shape[0]))
    plt.xticks(range(mt.shape[1]))
    plt.colorbar()
    plt.savefig("./kldiv.png")

In [0]:
#warningを無視する
import warnings
warnings.filterwarnings('ignore')

# LDA
num_topics = 20

"""
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus_tfidf,
                                            id2word=dictionary,
                                            num_topics=num_topics,
                                            random_state=0)
"""


lda_model = gensim.models.LdaModel(corpus=corpus, id2word=dictionary,
                             num_topics=num_topics, minimum_probability=0.001,
                             passes=20, update_every=0, chunksize=10000, random_state=1)


In [15]:
"""
# KL divergence
plot_distance_matrix(lda_model)

# Visualize
ncols = math.ceil(num_topics/2)
nrows = math.ceil(lda_model.num_topics/ncols)
fig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=(15,3))
axs = axs.flatten()

def color_func(word, font_size, position, orientation, random_state, font_path):
    return 'darkturquoise'

for i, t in enumerate(range(lda_model.num_topics)):

    x = dict(lda_model.show_topic(t, 30))
    im = WordCloud(
        font_path=FONT,
        background_color='white',
        color_func=color_func,
        random_state=0
    ).generate_from_frequencies(x)
    axs[i].imshow(im)
    axs[i].axis('off')
    axs[i].set_title('Topic '+str(t))

plt.tight_layout()
plt.savefig("./visualize.png")
"""

'\n# KL divergence\nplot_distance_matrix(lda_model)\n\n# Visualize\nncols = math.ceil(num_topics/2)\nnrows = math.ceil(lda_model.num_topics/ncols)\nfig, axs = plt.subplots(ncols=ncols, nrows=nrows, figsize=(15,3))\naxs = axs.flatten()\n\ndef color_func(word, font_size, position, orientation, random_state, font_path):\n    return \'darkturquoise\'\n\nfor i, t in enumerate(range(lda_model.num_topics)):\n\n    x = dict(lda_model.show_topic(t, 30))\n    im = WordCloud(\n        font_path=FONT,\n        background_color=\'white\',\n        color_func=color_func,\n        random_state=0\n    ).generate_from_frequencies(x)\n    axs[i].imshow(im)\n    axs[i].axis(\'off\')\n    axs[i].set_title(\'Topic \'+str(t))\n\nplt.tight_layout()\nplt.savefig("./visualize.png")\n'

In [16]:
# WordCloud2
"""
plt.figure(figsize=(30, 30))
for t in range(lda_model.num_topics):
    plt.subplot(5, 4, t+1)
    x = dict(lda_model.show_topic(t, 200))
    im = WordCloud(font_path=FONT).generate_from_frequencies(x)
    plt.imshow(im)
    plt.axis("off")
    plt.title("Topic #" + str(t))

plt.savefig("./visualize2.png")
"""

'\nplt.figure(figsize=(30, 30))\nfor t in range(lda_model.num_topics):\n    plt.subplot(5, 4, t+1)\n    x = dict(lda_model.show_topic(t, 200))\n    im = WordCloud(font_path=FONT).generate_from_frequencies(x)\n    plt.imshow(im)\n    plt.axis("off")\n    plt.title("Topic #" + str(t))\n\nplt.savefig("./visualize2.png")\n'

In [17]:
# pyLDAvis
"""
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(vis, './pyldavis_output.html')
"""

"\nvis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, sort_topics=False)\npyLDAvis.save_html(vis, './pyldavis_output.html')\n"

In [18]:
# Vis t-SNE
"""
vis_tsne = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, mds='tsne', sort_topics=False)
vis_tsne

# save as html
pyLDAvis.save_html(vis_tsne, 'pyldavis_output_tsne.html')
"""

"\nvis_tsne = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, mds='tsne', sort_topics=False)\nvis_tsne\n\n# save as html\npyLDAvis.save_html(vis_tsne, 'pyldavis_output_tsne.html')\n"

In [74]:
# output tags in topics
for x in lda_model.show_topics(-1, 20):
    print(x)
print(type(x))

(0, '0.007*"秘境" + 0.004*"ランタン" + 0.003*"軽井沢" + 0.003*"会場" + 0.003*"自動車" + 0.003*"沿線" + 0.003*"無人" + 0.003*"上り" + 0.003*"カカオ" + 0.003*"ハイウェイ" + 0.003*"タイ" + 0.003*"出雲" + 0.003*"長崎" + 0.003*"スウェーデン" + 0.003*"六甲" + 0.003*"古墳" + 0.002*"ホーム" + 0.002*"高速" + 0.002*"オアシス" + 0.002*"列車"')
(1, '0.006*"工業" + 0.005*"地帯" + 0.004*"神楽坂" + 0.004*"スウェーデン" + 0.004*"白味噌" + 0.004*"プラザ" + 0.004*"田園" + 0.004*"バーガー" + 0.004*"フィンランド" + 0.003*"群馬" + 0.003*"両国" + 0.003*"教会" + 0.003*"築地" + 0.003*"大分" + 0.003*"鳴子" + 0.002*"鳥取" + 0.002*"佐世保" + 0.002*"山梨" + 0.002*"縁切り" + 0.002*"ご利益"')
(2, '0.007*"倉敷" + 0.005*"梅林" + 0.004*"軽井沢" + 0.003*"松本" + 0.003*"金沢" + 0.003*"新潟" + 0.003*"見頃" + 0.002*"スペイン" + 0.002*"たぬき" + 0.002*"美観" + 0.002*"教会" + 0.002*"手土産" + 0.002*"銭湯" + 0.002*"沖縄" + 0.002*"草津" + 0.002*"宇治" + 0.002*"わん" + 0.002*"安曇" + 0.002*"イルミネーション" + 0.002*"マーケット"')
(3, '0.005*"ポートランド" + 0.004*"沖縄" + 0.004*"山形" + 0.003*"金沢" + 0.003*"北斎" + 0.003*"札幌" + 0.003*"列車" + 0.003*"博多" + 0.002*"スキー" + 0.002*"スウェーデン" + 0.002*"福岡" + 0.0

In [45]:
#0~19トピックの頻出語を20個抽出する

for i in range(0, 20):
    exec("t_words" + str(i) + "= []")
    exec("t_words%d = lda_model.show_topic(i, 20)" % (i))

print(t_words2)

[('倉敷', 0.007087891), ('梅林', 0.004809024), ('軽井沢', 0.0038061435), ('松本', 0.003443451), ('金沢', 0.00310948), ('新潟', 0.002595833), ('見頃', 0.0025816269), ('スペイン', 0.0023173715), ('たぬき', 0.0022618936), ('美観', 0.0022483035), ('教会', 0.002168357), ('手土産', 0.002135517), ('銭湯', 0.002129149), ('沖縄', 0.002121299), ('草津', 0.0020979205), ('宇治', 0.0020792256), ('わん', 0.0020321526), ('安曇', 0.0019751245), ('イルミネーション', 0.0019698914), ('マーケット', 0.0019631684)]


In [0]:
#無関係なワード削除
def format_text(text):

    text=re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-…]+', "", text)
    #text=re.sub('投稿', "", text)
    #text=re.sub(r'\W', "", text)#英数字以外
    #text=re.sub(r'\A', "", text)#文字列の先頭
    #text=re.sub(r'\Z', "", text)#文字列の最後尾
    text=re.sub(r'[!-~]', "", text)#半角記号,数字,英字
    text=re.sub(r'[︰-＠]', "", text)#全角記号
    #text=re.sub('\n', " ", text)#改行文字

    return text

In [69]:
#文章から数字等を削除する

for i in range(0,20):
    exec("tt_words%d = ','.join(map(str, t_words%d))" % (i, i))
    exec("topic_words" + str(i) + "= []")
    exec("tw%d = format_text(tt_words%d)" % (i, i))
    exec("topic_words%d = tw%d.split()" % (i, i))

print(topic_words2)

['倉敷', '梅林', '軽井沢', '松本', '金沢', '新潟', '見頃', 'スペイン', 'たぬき', '美観', '教会', '手土産', '銭湯', '沖縄', '草津', '宇治', 'わん', '安曇', 'イルミネーション', 'マーケット']


In [43]:
#各トピックの特徴語をカウントして特徴ベクトルを作る

for i in range(0, 20):
    exec("tmp += [dictionary.doc2bow(topic_words%d)]"% (i))

print(tmp)

dense = [list(matutils.corpus2dense([tmp[i]], num_terms=len(dictionary)).T[0]) for i in range(0, 20)]

print(dense)

[[(787, 1), (1566, 1), (1659, 1), (1831, 1), (1856, 1), (1903, 1), (1959, 1), (2040, 1), (2244, 1), (2386, 1), (2535, 1), (2711, 1), (3238, 1), (3548, 1), (3681, 1), (4054, 1), (5917, 1), (8101, 1), (8177, 1), (9185, 1)], [(94, 1), (98, 1), (306, 1), (363, 1), (401, 1), (1294, 1), (1300, 1), (1677, 1), (2021, 1), (2087, 1), (2330, 1), (2761, 1), (2833, 1), (3431, 1), (3548, 1), (4186, 1), (5224, 1), (6613, 1), (8450, 1), (8538, 1)], [(28, 1), (319, 1), (472, 1), (573, 1), (633, 1), (774, 1), (821, 1), (859, 1), (1024, 1), (1809, 1), (1921, 1), (1947, 1), (2364, 1), (3548, 1), (3575, 1), (4417, 1), (5068, 1), (5880, 1), (7221, 1), (8954, 1)], [(70, 1), (374, 1), (633, 1), (718, 1), (857, 1), (911, 1), (981, 1), (1365, 1), (1367, 1), (1420, 1), (1461, 1), (2265, 1), (2386, 1), (2833, 1), (3239, 1), (4129, 1), (4458, 1), (5917, 1), (6565, 1), (6790, 1)], [(325, 1), (392, 1), (412, 1), (561, 1), (675, 1), (718, 1), (827, 1), (981, 1), (1300, 1), (1420, 1), (1568, 1), (1891, 1), (2040, 1), 

In [0]:
#機械学習

from sklearn.ensemble import RandomForestClassifier

# 1個目トピック0,,,,,,,の特徴ベクトル
data_train =  dense

# 正解のラベル
label_train = [i for i in range(0, 20)]  # トピック1~19に分類

estimator = RandomForestClassifier()

# 学習させる
estimator.fit(data_train, label_train)

In [0]:
# 予測
label_predict = estimator.predict(data_train)
print(label_predict)

In [0]:
from sklearn.model_selection import train_test_split

# 6割を学習用、 4割を試験用にする
data_train_s, data_test_s, label_train_s, label_test_s = train_test_split(data_train, label_train, test_size=0.3)

# 学習用に切り出したやつだけで学習
estimator.fit(data_train_s, label_train_s)

# 予測。正解が分かってる場合は、predict関数じゃなくてこうやると、正解率出してくれる
print(estimator.score(data_test_s, label_test_s))